# Filtering the rolled and latent mates 

present in the finger position annotation in the SD302h subset.

In [ ]:
import os
import sys
import pandas as pd

sys.path.append(os.path.abspath('../src'))
from utils.file import list_files, make_dirs, get_file_name


## NIST SD302h path

In [ ]:
latent_dir = r'H:\Workspace\KSIP\02_Database\00_ORIGINAL\NIST_SD302\sd302e\images\latent\png'
csv_path = r'H:\Workspace\KSIP\02_Database\00_ORIGINAL\NIST_SD302\sd302h\ebts\latent\finger_positions.csv'

In [ ]:
data = pd.read_csv(csv_path)
display(data.head())
display(data.tail())
data.count()


In [ ]:
data.describe()

## Filter Nan

In [ ]:
data = data.dropna()
display(data.count())
data.describe()


In [ ]:
ct = 0
finger_pos = {'filename':[]}
for d in data.iterrows():
    d = d[1]
    filename = d.filename
    if 'X' in filename:
        continue
    ct += 1
print(ct)
    

## Filter impression

In [ ]:
ct = 0
finger_pos = {'filename':[]}
for d in data.iterrows():
    d = d[1]
    filename = d.filename
    # print(filename)
    filename_split = filename.split("_")
    subject = filename_split[0]
    # img_path = os.path.join(img_dir, '%s/%s'%(subject, filename.replace('.lffs','').replace('1000PPI','').replace('-1_1', '_1')))
    # print(img_path)
    if '-2' in filename or '-3' in filename or '-4' in filename:
        continue
    finger_pos['filename'].append('%s_x_%02d.png'%(subject,d.fgp))
    ct += 1
print(ct)
    

In [ ]:
df_finger_pos = pd.DataFrame(finger_pos)
df_finger_pos

In [ ]:
img_roll_dir = r'H:\Workspace\KSIP\02_Database\00_ORIGINAL\NIST_SD302\sd302b\images\baseline\U\500\roll\png'
img_roll_dir = r'H:\Workspace\KSIP\02_Database\00_ORIGINAL\NIST_SD302\sd302b\images\baseline\V\500\roll\png'

name = set()
ct = 0
for d in df_finger_pos.iterrows():
    d = d[1]
    filename = d.filename
    print(filename)
    subject = filename_split[0]
    # img_path = os.path.join(img_roll_dir, '%s'%(filename.replace('x','U_500_roll')))
    img_path = os.path.join(img_roll_dir, '%s'%(filename.replace('x','V_500_roll')))
    # print(img_path)
    
    if os.path.exists(img_path):
        ct += 1
        # name.add('%s'%(filename.replace('x','U_500_roll')))
        name.add('%s'%(filename.replace('x','V_500_roll')))
print(ct)
print(len(name))
    